In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from pandas import DataFrame as df
from Data_setup_AEMEL_NewCell_CP import data_set
import os
from galvani import BioLogic

fig = go.Figure()
colors = px.colors.qualitative.Safe

for i in data_set:
    file = data_set[i]['data_PEIS']
    if not os.path.exists(file):
        print("File not found: " + file)
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file, encoding='ansi', sep="\t", decimal=get_decimal_separator(file),  skiprows=range(headerlength))
        
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        
    
    real = data['Re(Z)/Ohm']
    imaginary = data['-Im(Z)/Ohm']
       
   
    fig.add_trace(go.Scatter(
        x=real,
        y=imaginary,
        name= data_set[i]['label'],
        line_color = colors[data_set[i]['color_index']],
    )
           
    )
fig.update_xaxes(range=[0, 2],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[0, 2],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    
    xaxis_title="Re(Z) (ohm)",
    yaxis_title="-Im(Z) (ohm)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    width = 500,
    height = 500,
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0,
    font=dict(
        family="Times New Roman, monospace",
        size=22,
        color="dimgrey",
    ),
    )
)
fig.show()
fig.write_image('PEIS_firstpaperall.jpeg')
fig.write_image('PEIS_firstpaperall.svg')


File not found: 
